In [1]:
import matplotlib.pyplot as plt
import numpy as np
from skopt import gp_minimize

In [2]:
# res = gp_minimize(loss_fn,                  # the function to minimize
#                   [[0,1] for _ in range(len(dims))],      # the bounds on each dimension of x
#                   acq_func="EI",      # the acquisition function
#                   n_calls=150,         # the number of evaluations of f
#                   n_random_starts=3,  # the number of random initialization points
#                   noise=0.1**2,       # the noise level (optional)
#                   n_jobs=5,
#                   random_state=1234)   # the random seed

In [2]:
a = [1, 4, 5, 3, 9]
np.flip(np.sort(a))

array([9, 5, 4, 3, 1])

In [3]:
from assignment import AssignmentGame
e = 2*[0] + 2*[0.15] + 6*[0.3]
c = [1 for _ in range(10)]
Q = 300
game = AssignmentGame(grid_size=30, max_capacity=50, emissions_KM=e, costs_KM=c, Q=Q)
K = 500
game.reset(num_packages = K)

{}

In [4]:
f = lambda x : -game.step(x, 1)[0]
res = gp_minimize(f,                  # the function to minimize
                  [[0,1] for _ in range(K)],      # the bounds on each dimension of x
                  acq_func="EI",      # the acquisition function
                  n_calls=200,         # the number of evaluations of f
                  n_random_starts=3,  # the number of random initialization points
                  noise=0.1**2,       # the noise level (optional)
                  n_jobs=5,
                  x0=[1 for _ in range(K)],#np.ones(K, dtype = int),
                  random_state=1234)   # the random seed

In [1]:
plt.plot(res['func_vals'])

NameError: name 'plt' is not defined

In [46]:
np.argmin(res['func_vals'])

177

In [ ]:
import numpy.random as rd
def rand_neighbor(solution : np.ndarray, nb_changes = 1) :
    """
    Generates new random solution.
    :param solution: the solution for which we search a neighbor
    :param nb_changes: maximum number of the changes alowed
    :return: returns a random neighbor for the solution
    """
    new_solution = solution.astype(bool)
    i = rd.choice(len(new_solution), nb_changes, replace=False)
    new_solution[i] = ~new_solution[i]
    return new_solution.astype(int)

def recuit(loss_fn, T_init, T_limit, lamb = .99, var = False, id = 0, log = True, H = 500) :
    """
    This function finds a solution for the steiner problem
        using annealing algorithm
    :param game: the assignment game
    :param T_init: the initial temperature
    :param T_limit: the lowest temperature allowed
    :return: the solution found and the evolution of the best evaluations
    """
    best = np.ones(len(dims), dtype=int)
    solution = best.copy()
    T = T_init
    eval_best = loss_fn(solution)
    eval_solution = eval_best
    m = 0
    list_best_costs = [eval_best]
    flag100 = True

    while(T>T_limit):
        sol = rand_neighbor(solution)
        eval_sol = loss_fn(sol)
        
        if m%20 == 0 and log:
            print(20*'-')
            print(m)
            print('- searcher ', id)
            print('temperature : ', T)
            print('cost : ', eval_sol)
            print('best cost : ', eval_best)
        if eval_sol < eval_best :
            best = sol.copy()
            eval_best = eval_sol
            
        if eval_sol < eval_solution :
            prob = 1
        else :
            prob = np.exp((eval_best - eval_sol)/T)
        rand = rd.random()
        if rand <= prob :
            solution = sol
            eval_solution = eval_sol
        list_best_costs.append(eval_best)
        T *= lamb
        m += 1
        if m >= H:
            break
        
        if(var and flag100 and T<=100):
            flag100 = False
            lamb = .999
        #print(T)

    print(f'm ={m}')
    print(eval_best)
    return best, list_best_costs


In [ ]:
best = recuit(f, 20000, 2, )